In [1]:
import os
import json
import csv

from tqdm import tqdm
import pandas as pd
import tweepy

In [2]:
with open('keys.json', 'rb') as f:
    keys = json.load(f)

In [3]:
auth = tweepy.OAuthHandler(
    keys['API_key'], 
    keys['API_key_secret']
)
auth.set_access_token(
    keys['access_token'], 
    keys['access_token_secret']
)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [34]:
int(len(tweet_ids) - len(tweet_ids) % 100)

50000000

In [35]:
def hydrate(file_id):
    
    tweet_ids = pd.read_csv(os.path.join(
        'data',
        'dataverse_files',
        f'election-filter{file_id}.txt'
    ), header = None)[0].values
    
    l = len(tweet_ids)
    rounded = int(l - l % 100)
    tweet_ids = tweet_ids[:rounded].reshape(l // 100, 100)
    
    headers1 = [
        'author_id',
        'author_screen_name',
        'status_id',
        'created_at',
        'body',
        'lang',
        'favorite_count',
        'retweet_count'
    ]
    
    headers2 = [
        'user_id',
        'user_screename',
        'follower_count',
        'verified',
        'statuses_count'
    ]

    with open(os.path.join('data', f'tweets_{file_id}.csv'), 'w', newline = '', encoding = 'utf-8') as csv_file_tweets:
        writer1 = csv.DictWriter(csv_file_tweets, fieldnames = headers1)
        writer1.writeheader()
        with open(os.path.join('data',f'users_{file_id}.csv'), 'w', newline = '') as csv_file_users:
            writer2 = csv.DictWriter(csv_file_users, fieldnames = headers2)
            writer2.writeheader()
            for id_list in tqdm(tweet_ids):
                try:
                    statuses = api.statuses_lookup(
                        list(id_list)
                    )            
                except tweepy.error.TweepError:
                    continue
                else:
                    for status in statuses:
                        author = status.user

                        author_id = author.id
                        author_screen_name = author.screen_name

                        status_id = status.id
                        created_at = status.created_at

                        if hasattr(status, 'retweeted_status'):
                            try:
                                body = status.retweeted_status.extended_tweet['full_text']
                            except AttributeError:
                                body = status.retweeted_status.text
                        else:
                            try:
                                body = status.extended_tweet['full_text']
                            except AttributeError:
                                body = status.text

                        lang = status.lang

                        favorite_count = status.favorite_count
                        retweet_count = status.retweet_count

                        writer1.writerow({
                            'author_id' : author_id,
                            'author_screen_name' : author_screen_name,
                            'status_id' : status_id,
                            'created_at' : created_at,
                            'body' : body,
                            'lang' : lang,
                            'favorite_count' : favorite_count,
                            'retweet_count' : retweet_count
                        })

                        writer2.writerow({
                            'user_id' : author_id,
                            'user_screename' : author_screen_name,
                            'follower_count' : author.followers_count,
                            'verified' : author.verified,
                            'statuses_count' : author.statuses_count
                        })


In [36]:
hydrate(2)

  0%|          | 5/500000 [00:05<152:25:35,  1.10s/it]


KeyboardInterrupt: 